In [1]:
import matplotlib.pyplot as plt
import astropy.units as u
import numpy as np
from ClusterModel import model
from ClusterTools import cluster_profile

In [2]:
clust = model.Cluster()

                      ___   __    ___   __   __                      
                     / __) / _\  / __) /  \ (  )                     
                    ( (__ /    \( (_ \(  O )/ (_/\                   
                     \___)\_/\_/ \___/ \__/ \____/                   
   Cluster Atmosphere modeling for Gamma-ray Observations Libraries  
---------------------------------------------------------------------
                                                                     
Setting gNFW Planck coll. (2013) UPP.


In [3]:
clust.pressure_gas_model

Getting the gas electron pressure profile model value


{'P_0': <Quantity 0.02519719 keV / cm3>,
 'a': 1.33,
 'b': 4.13,
 'c': 0.31,
 'c500': 1.81,
 'name': 'GNFW',
 'r_p': <Quantity 855.62300911 kpc>}

In [4]:
r, p = clust.get_pressure_gas_profile()
r, n = clust.get_density_gas_profile()
r, t = clust.get_temperature_gas_profile()
r, ncr = clust.get_density_crp_profile()
r, b = clust.get_magfield_profile()
r,x = clust.get_crp_to_thermal_energy_profile()

In [5]:
from ClusterModel import model_modpar

In [6]:
mp = model_modpar.Modpar()

In [7]:
clust.set_pressure_gas_gNFW_param()

Setting gNFW Planck coll. (2013) UPP.


In [8]:
clust.output_dir = '/Users/adam/Desktop/testdir'

Setting output_dir value


In [9]:
clust.EBL_model = 'gilmore'
clust.plot()

Setting EBL_model value


/Users/adam/Project/anaconda3/envs/cta/lib/python2.7/site-packages/astropy/units/quantity.py:641: RuntimeWarning: invalid value encountered in greater
  *arrays, **kwargs)


!!! WARNING: XSPEC_table.txt not generated, skip Xray flux and Sx


/Users/adam/Project/anaconda3/envs/cta/lib/python2.7/site-packages/numpy/ma/core.py:6666: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)


!!! WARNING: XSPEC_table.txt not generated, skip sx_map


In [10]:
r,x = clust.get_crp_to_thermal_energy_profile()